In [3]:
!python main.py --base configs/faceshq_vqgan.yaml -t True --gpus 0;

python: can't open file 'main.py': [Errno 2] No such file or directory


In [1]:
!ls

License.txt		reconstruction_usage_update.ipynb
README.md		reset.ipynb
assets			scripts
configs			server
data			setup.py
dataset.py		taming
dataset_tool.py		taming_transformers.egg-info
download_dataset.ipynb	test.ipynb
environment.yaml	tfutil.py
main.py


In [4]:
!export XDG_CACHE_HOME='/data/vqvae'

In [ ]:
!python main.py --base configs/imagenet_vqgan.yaml -t True --gpus 0;

Running on GPUs 0
Working with z of shape (1, 256, 16, 16) = 65536 dimensions.
loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth
VQLPIPSWithDiscriminator running with hinge loss.
/opt/conda/envs/taming/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: ModelCheckpoint(save_last=True, monitor=None) is a redundant configuration. You can save the last checkpoint with ModelCheckpoint(save_top_k=None, monitor=None).
  warnings.warn(*args, **kwargs)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
Preparing dataset ILSVRC2012_train in /data/vqgan/autoencoders/data/ILSVRC2012_train
Torrent name: ILSVRC2012_img_train.tar, Size: 147897.48MB
Checking pieces on disk...
  0%|▏                                     | 280/70524 [00:00<02:23, 490.28it/s]